In [ ]:
# 
# Copyright (C) 2019-2021 vdaas.org vald team <vald@vdaas.org>
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# 

# Vald Similarity Search using chiVe Dataset

---
※***このnotebookは, 既に[Get Started](https://vald.vdaas.org/docs/tutorial/get-started/)を完了し, Valdの環境構築が完了した方を対象としています.  
まだValdの環境構築がお済みでない方は, 先に[Get Started](https://vald.vdaas.org/docs/tutorial/get-started/)を行うことを推奨します.  
また, データセットとしてchiVeを利用する場合, Vald Agentのdimensionを300に, distance_typeをcosineにすることを推奨するため, [sample-values.yaml](https://github.com/vdaas/vald-demo/blob/main/chive/sample-values.yaml)を用いる or 値を修正したvalues.yamlを用いてValdの構築を行ってください.***  
- *dimension: 300, distance_type: consieの修正例 ([path/to/helm/values.yaml](https://github.com/vdaas/vald/blob/master/example/helm/values.yaml#L45-L49))*:
```yaml
agent:
  ngt:
    dimension: 300
    distance_type: cos
```
---

このnotebookの目的は, vald-python-clientを通じて, Valdの基礎的な動作であるInsert/Search/Update/Removeを体験し, 近似近傍探索を用いた検索の一例を体験することです.  
今回, 検索を行うためのデータセットとして, 日本語単語ベクトルのデータセットである[chiVe](https://github.com/WorksApplications/chiVe)を利用しています.

notebookの概要は以下の通りです:
- Preprocess
  - Install packages
  - Import dependencies
  - Prepare the vector data with chiVe
- Similarity Search with vald-client-python
  - Create gRPC channel
  - Insert/Search/Update/Remove
- Advanced
  - Word Analogies
  
それでは, Valdを利用した近似近傍探索による検索を体験してみましょう!!

---

## Preprocess

Valdを利用するにあたって必要なパッケージやベクトルデータを準備します.

### Install packages

※*動作環境に応じてパッケージのインストールを行ってください.*

In [ ]:
!pip install grpcio pymagnitude vald-client-python

### Import dependencies

notebookを実行するに当たり, 必要なパッケージをインポートします.

In [ ]:
import grpc
import io
import os
import pandas as pd

from pymagnitude import Magnitude
from tqdm.notebook import tqdm
from vald.v1.payload import payload_pb2
from vald.v1.vald import (insert_pb2_grpc,
                          object_pb2_grpc,
                          remove_pb2_grpc,
                          search_pb2_grpc,
                          update_pb2_grpc)

### Prepare the vector data with [chiVe](https://github.com/WorksApplications/chiVe)

このnotebookでは, 日本語単語ベクトルとして[chiVe](https://github.com/WorksApplications/chiVe)を用いるため, 予め必要なデータをダウンロードしておくことを推奨します.
```
curl "https://sudachi.s3-ap-northeast-1.amazonaws.com/chive/chive-1.2-mc90.magnitude" -o "chive-1.2-mc90.magnitude"
```

データを読み込み, サンプルとなるクエリを用いてベクトルを表示します.

In [ ]:
# NOTE: "___" -> "/path/to/chive-1.2-mc90.magnitude"
vectors = Magnitude("___")
"テスト" in vectors, vectors.query("テスト")

---

## Similarity Search with vald-client-python

Valdの基礎的な動作であるInsert/Search/Update/Removeを実行すると共に, 前項で準備したベクトルデータを用いて近似近傍探索による検索を行います.

### Create gRPC channel

gRPCによる通信を行うため, Valdが動作している各環境に応じて, 必要なエンドポイントを記述し, channelを作成します.

In [ ]:
# NOTE: "___" -> "{host}:{port}"
channel = grpc.insecure_channel("___")

### Insert

初めに, Valdにデータを入れるため, Insertを行います.  
Insertを行うため, 先程作成したchannelを用いてInsert用のstubを作成します.

In [ ]:
# create stub
istub = insert_pb2_grpc.InsertStub(channel)

次に, Insert命令を用いてValdにデータ(id="test")を1件Insertし, 正常に動作が完了するか確認します.

In [ ]:
ivec = payload_pb2.Object.Vector(id="test", vector=vectors.query("テスト"))
icfg = payload_pb2.Insert.Config(skip_strict_exist_check=True)
ireq = payload_pb2.Insert.Request(vector=ivec, config=icfg)

istub.Insert(ireq)

1件のデータ(id="test")のInsertの動作確認が完了次第, 100,000件のデータをValdにInsertします.  
ここで, Insertの時間短縮のため, 複数のデータを用いてInsertを行うMultiInsertを使用しています.

In [ ]:
# Insert 100*1000 vector
count = 100
length = 1000

for c in tqdm(range(count)):
    ireqs = []
    for key, vec in vectors[c*length:(c+1)*length]:
        ivec = payload_pb2.Object.Vector(id=key, vector=vec)
        icfg = payload_pb2.Insert.Config(skip_strict_exist_check=True)
        ireq = payload_pb2.Insert.Request(vector=ivec, config=icfg)
        ireqs.append(ireq)    
    imreq = payload_pb2.Insert.MultiRequest(requests=ireqs)
    istub.MultiInsert(imreq)

### Search

次に, 先程Insertしたデータを使用してSearchを行います.  
Insert時と同様, Search用のstubを作成します.

In [ ]:
# create stub
sstub = search_pb2_grpc.SearchStub(channel)

SearchのRequestを作成し, stubを用いて, __"テスト"__に類似したテキストを検索します.

※*検索結果が0件または極端に少ない場合, Valdの自動Indexingにより検索結果が返却されていない可能性があるため, Indexing完了のため数秒待機し, 再度Searchを行ってください.*

In [ ]:
svec = vectors.query("テスト")
scfg = payload_pb2.Search.Config(num=10, radius=-1.0, epsilon=0.01, timeout=3000000000)
sreq = payload_pb2.Search.Request(vector=svec, config=scfg)

response = sstub.Search(sreq)
pd.DataFrame(
    [(result.id, result.distance) for result in response.results],
    columns=["id", "distance"])

また, Valdは既に入力済みのデータに対して, idに紐づくベクトルを用いて検索を行うSearch By IDにも対応しています.  
以下で, 既にInsert済みのid="test"に紐づくベクトルを使用し, 近似近傍探索による検索を行います.

In [ ]:
# Search By ID
sireq = payload_pb2.Search.IDRequest(id="test",config=scfg)

response = sstub.SearchByID(sireq)
pd.DataFrame(
    [(result.id, result.distance) for result in response.results],
    columns=["id", "distance"])

### Update

ここでは, idに紐づくInsert済みのデータを更新するUpdateを行います.  
Updateを行うため, 同様にstubを作成します.

In [ ]:
# create stub
ustub = update_pb2_grpc.UpdateStub(channel)

id="test"に紐づくデータを__"テスト"__から__"test"__のベクトルに更新します.

In [ ]:
uvec = payload_pb2.Object.Vector(id="test", vector=vectors.query("test"))
ucfg = payload_pb2.Update.Config(skip_strict_exist_check=True)
ureq = payload_pb2.Update.Request(vector=uvec, config=ucfg)

ustub.Update(ureq)

Updateの確認のため, id="test"に紐づくデータに対して検索を行い, __"テスト"__の結果と異なることを確認します.

※*Insert済みのデータによっては, 同様の結果となる場合もあります.*  
※*Searchの際と同様に, Indexingなどのタイミングによっては値が変更されていない可能性があるため, 時間をおいて再度検索を行ってください.*

In [ ]:
# Search By ID
sireq = payload_pb2.Search.IDRequest(id="test", config=scfg)

response = sstub.SearchByID(sireq)
pd.DataFrame(
    [(result.id, result.distance) for result in response.results],
    columns=["id", "distance"])

### Remove

最後に, 入力されたデータを削除するRemoveを行います.  
Remove用のstubを作成します.

In [ ]:
# create stub
rstub = remove_pb2_grpc.RemoveStub(channel)

id="test"に紐づくデータを削除します.

In [ ]:
rid = payload_pb2.Object.ID(id="test")
rcfg = payload_pb2.Remove.Config(skip_strict_exist_check=True)
rreq = payload_pb2.Remove.Request(id=rid, config=rcfg)

rstub.Remove(rreq)

データが削除されたかどうかを確認するため, Existを用いてデータが存在するかどうかをチェックします(データが存在しない場合Errorを返します).

In [ ]:
# Exists
ostub = object_pb2_grpc.ObjectStub(channel)

oid = payload_pb2.Object.ID(id="test")
try:
    ostub.Exists(oid)
except grpc._channel._InactiveRpcError as _:
    print("vector is not found")

以上が, Valdの基礎的な動作であるInsert/Search/Update/Removeを用いた検索の例です.

---

## Advanced

近似近傍探索を用いたテキスト検索の実験的な例として, 以下の内容を行います.

- Word Analogies

### Word Analogies

単語のベクトル表現を用いて, 加法/減法を行い, 意味的に類似した単語を検索します.  
例として, ”王"-"男"+"女"="女王"となるベクトル表現が得られ, "女王"に意味的に類似した単語が結果に含まれることを期待するテキスト検索を以下を示します. 

In [ ]:
svec = vectors.query("王") - vectors.query("男") + vectors.query("女")
scfg = payload_pb2.Search.Config(num=10, radius=-1.0, epsilon=0.01, timeout=3000000000)
sreq = payload_pb2.Search.Request(vector=svec, config=scfg)
response = sstub.Search(sreq)

pd.DataFrame(
    [(result.id, result.distance) for result in response.results],
    columns=["id", "distance"])

また, 上記とは異なる例も示します(ref: [fastText tutorial#word-analogies](https://fasttext.cc/docs/en/unsupervised-tutorial.html#word-analogies)).

In [ ]:
svec = vectors.query("psx") - vectors.query("sony") + vectors.query("nintendo")
scfg = payload_pb2.Search.Config(num=10, radius=-1.0, epsilon=0.01, timeout=3000000000)
sreq = payload_pb2.Search.Request(vector=svec, config=scfg)
response = sstub.Search(sreq)

pd.DataFrame(
    [(result.id, result.distance) for result in response.results],
    columns=["id", "distance"])

---

以上で__"Vald Similarity Search using chiVe Dataset"__ notebookは終了です.  
Valdに興味を持っていただきありがとうございました.

更に詳しく知りたい方は, Githubやofficial web siteをご活用ください:
- https://github.com/vdaas/vald
- https://vald.vdaas.org/